In [1]:
import pandas as pd
import numpy as np
from load_data import IHDP_Dataset
from model import CFR
from optimize import MMD

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
# データの読み込み
train_D = IHDP_Dataset(train=True)
train_loader = DataLoader(train_D, batch_size=32, shuffle=True)
len(train_loader)

21

In [8]:
class MMD(torch.nn.Module):
    '''emprical MMD
    '''
    def __init__(self):
        super().__init__()
        
    def forward(self, X, Y, sigma=10):
        # ガウスカーネルを採用する。
        # | x_i - x_j |^2 = x_i^T x_i + x_j^T x_j - 2x_i^T x_j
        # の値を算出する
        print(Y.shape)
        xx, xy, yy = torch.mm(X, X.T), torch.mm(X, Y.T), torch.mm(Y, Y.T)
        expand_diag_xx = xx.diag().expand_as(xx)
        expand_diag_yy = yy.diag().expand_as(yy)
        
        dxx = expand_diag_xx.T + expand_diag_xx - 2 * xx
        print(expand_diag_xx.T.shape)
        print(expand_diag_yy.shape)
        dxy = expand_diag_xx.T + expand_diag_yy - 2 * xy
        dyy = expand_diag_yy.T + expand_diag_yy - 2 * yy
        
        XX = torch.exp(-0.5*dxx/sigma)
        XY = torch.exp(-0.5*dxy/sigma)
        YY = torch.exp(-0.5*dyy/sigma)
        return torch.mean(XX + YY - 2 * XY)

In [9]:
# 訓練
ETA = 1e-2
alpha = 0.1

net = CFR(feature_dim=train_D.x.shape[1])
optimizer_w = Adam(net.representation.parameters(), lr=ETA)
optimizer_v = Adam(net.hypothesis.parameters(), lr=ETA, weight_decay=0.9)

mmd = MMD()
mse = torch.nn.MSELoss()

torch.random.manual_seed(0)
for batch in train_loader:
    optimizer_w.zero_grad()
    optimizer_v.zero_grad()
    x_batch, t_batch = batch['x'], batch['t']
    r_batch, y_hat_batch = net(x_batch, t_batch)
    
    # IPM項の勾配
    treat_idx = torch.where(t_batch == 1)
    control_idx = torch.where(t_batch == 0)
    # NOTE: Wassersteinを使うときはこの行を変更する。
    # FIXME: 介入群と統制群のサンプル数が一致せずエラーが起きている。
    #        -> MMDの計算の実装の行列の和をとる部分において、形が一致せずエラー
    #        -> デザイン行列は、(n, d)?それとも(d, n)
    ipm_term = alpha * mmd(r_batch[control_idx], r_batch[treat_idx])
    ipm_term.backward()
        
    # FIXME: 重み係数行列をLに乗算する必要がある。
    L = mse(y_hat_batch, batch['yf'])
    L.backward()
    optimizer_w.step()
    optimizer_v.step()

torch.Size([6, 200])
torch.Size([26, 26])
torch.Size([6, 6])


RuntimeError: The size of tensor a (26) must match the size of tensor b (6) at non-singleton dimension 1